In [3]:
import pandas as pd
import ast
import numpy as np
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors

In [5]:
annotation_df = pd.read_csv('150_papers_json_update.csv')
annotation_df

,first_num,id,text,memory,output,second_num
0,0,0_54,\t\t\t of 5 Downloaded from https://www.scienc...,"{""perovskite_composition"": ""Cs0.05FA0.85MA0.1P...","{""perovskite_composition"": ""Cs0.05FA0.85MA0.1P...",54
1,1,1_22,\t\t\t NAture PhotoNiCS | VOL 13 | JULY 2019 |...,"{""perovskite_composition"": null, ""electron_tra...","{""perovskite_composition"": null, ""electron_tra...",22
2,2,2_75,\t\t\t Nature eNerGY | VOL 6 | JANUARY 2021 | ...,"{""perovskite_composition"": ""(BA)2PbI 4"", ""elec...","{""perovskite_composition"": ""(BA)2PbI 4"", ""elec...",75
3,3,3_52,\t\t\t of 6 RESEARCH | REPORT Downloaded from ...,"{""perovskite_composition"": ""Cs0.05(MA0.10FA0.8...","{""perovskite_composition"": ""Cs0.05(MA0.10FA0.8...",52
4,4,4_26,"Proppe 1,2,10 , Andrew Johnston 2,10 , Sam T...","{""perovskite_composition"": null, ""electron_tra...","{""perovskite_composition"": ""(MAPbBr3)0.05(FAPb...",26
...,...,...,...,...,...,...
143,145,145_31,\t\t\t www.advancedsciencenews.com© 2018 WILEY...,"{""perovskite_composition"": ""Cs0.05(MA0.17FA0.8...","{""perovskite_composition"": ""Cs0.05(MA0.17FA0.8...",31
144,146,146_36,\t\t\t https://doi.org/10.1038/s41467-022-3420...,"{""perovskite_composition"": ""Cs0.05(MA0.05FA0.9...","{""perovskite_composition"": ""Cs0.05(MA0.05FA0.9...",36
145,147,147_41,Table 1 | PV parameters of the best-performed...,"{""perovskite_composition"": ""FAPbI 3"", ""electro...","{""perovskite_composition"": ""FAPbI 3"", ""electro...",41
146,148,148_26,"\t\t\t 15214095, 2020, 12, Downloaded from htt...","{""perovskite_composition"": ""Cs0.05FA0.85MA0.10...","{""perovskite_composition"": ""Cs0.05FA0.85MA0.10...",26


In [19]:
import pandas as pd
import json

# Load the dataset
file_path = "150_papers_json_update.csv"
annotation_df = pd.read_csv(file_path)

# List to store extracted data
expanded_data = []

for index, row in annotation_df.iterrows():
    try:
        row_dict = json.loads(row['output'])  # Convert JSON string to dictionary
    except (json.JSONDecodeError, TypeError):
        continue  # Skip rows where conversion fails

    # Extract common fields
    common_fields = {
        "first_num": row['first_num'],
        "perovskite_composition": row_dict.get("perovskite_composition"),
        "electron_transport_layer": row_dict.get("electron_transport_layer"),
        "hole_transport_layer": row_dict.get("hole_transport_layer"),
        "structure_pin_nip": row_dict.get("structure_pin_nip"),
    }

    # Extract test data
    for key, test_data in row_dict.items():
        if key.startswith("test_") and isinstance(test_data, dict):
            test_row = common_fields.copy()
            test_row["test"] = key  # Store test name
            test_row.update(test_data)  # Merge test details
            expanded_data.append(test_row)

# Convert extracted data to DataFrame
df_expanded = pd.DataFrame(expanded_data)
df_expanded.shape

(232, 18)

In [21]:
df_expanded

,first_num,perovskite_composition,electron_transport_layer,hole_transport_layer,structure_pin_nip,test,stability_type,passivating_molecule,humidity,temperature,time,control_pce,treated_pce,control_voc,treated_voc,efficiency_control,efficiency_tret,efficiency_cont
0,0,Cs0.05FA0.85MA0.1PbI3,C60,2PACz and Me-4PACz,PIN,test_1,ISOSL,4-chlorobenzenesulfonate (4Cl-BZS),None,65,1200,24,26.9,None,1.18,None,95,NaN
1,1,None,TinOxide,PTAA,PIN,test_1,ISOST,phenethylammonium,None,85,500,None,19.1,None,1.16,None,None,NaN
2,2,(BA)2PbI 4,tin dioxide,Spiro-OMeTAD,NIP,test_1,ISOSL,None,85,25,1620,22.39,24.35,None,1.185,None,98,58.6
3,2,(BA)2PbI 4,tin dioxide,Spiro-OMeTAD,NIP,test_1_2,ISOSD,None,85,85,1056,None,21.34,None,None,None,94,NaN
4,2,(BA)2PbI 4,tin dioxide,Spiro-OMeTAD,NIP,test_2,ISOSLT,None,None,25,1620,None,24.06,None,None,None,98,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,147,FAPbI 3,None,Spiro-OMeTAD,NIP,test_1,ISOSD,carbazole-triphenylamine and phenylammonium io...,85,85,1000,22.3,None,None,1.112,None,92.3,NaN
228,147,FAPbI 3,None,Spiro-OMeTAD,NIP,test_2,ISOSLT,carbazole-triphenylamine and phenylammonium io...,None,None,1100,None,24.7,None,None,None,94.6,66.6
229,148,Cs0.05FA0.85MA0.10Pb(I0.97Br0.03)3,None,Spiro-OMeTAD,NIP,test_1_2,ISOSLT,phenylethylammonium iodide,50-70,None,500,None,None,None,None,None,84,70
230,148,Cs0.05FA0.85MA0.10Pb(I0.97Br0.03)3,None,Spiro-OMeTAD,NIP,test_1,None,4-tert-butyl-benzylammonium iodide,None,None,None,21.2,22.7,1.091,1.122,None,95,NaN


In [23]:
df_expanded.to_csv('df_expanded.csv', index=False)

In [51]:
# Function to select columns
def select_data(df):
    # Convert PCE and VOC to numeric
    for col in ['control_pce', 'control_voc', 'treated_pce', 'treated_voc']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Drop rows where treated_pce or passivating_molecule is missing
    df = df.dropna(subset=['passivating_molecule'])

    return df

data = select_data(df_expanded)
data

,first_num,perovskite_composition,electron_transport_layer,hole_transport_layer,structure_pin_nip,test,stability_type,passivating_molecule,humidity,temperature,time,control_pce,treated_pce,control_voc,treated_voc,efficiency_control,efficiency_tret,efficiency_cont
0,0,Cs0.05FA0.85MA0.1PbI3,C60,2PACz and Me-4PACz,PIN,test_1,ISOSL,4-chlorobenzenesulfonate (4Cl-BZS),None,65,1200,24.0,26.90,NaN,1.180,None,95,NaN
1,1,None,TinOxide,PTAA,PIN,test_1,ISOST,phenethylammonium,None,85,500,NaN,19.10,NaN,1.160,None,None,NaN
6,5,BA2PbI 4,C60,Spiro-OMeTAD,NIP,test_1,None,iso-butylamine iodide,None,None,None,NaN,22.10,NaN,1.135,None,None,NaN
7,6,MAPbI 3,Titanium dioxide,Spiro-OMeTAD,None,test_1,ISOSD,phenylethylammonium iodide,None,None,1344,NaN,18.89,NaN,1.160,None,97,NaN
8,7,CsPbBr 3,C60,None,NIP,test_1,None,(ethylenedioxy)bis(ethylammonium),65,None,1000,NaN,NaN,NaN,NaN,None,90,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,147,FAPbI 3,None,Spiro-OMeTAD,NIP,test_1,ISOSD,carbazole-triphenylamine and phenylammonium io...,85,85,1000,22.3,NaN,NaN,1.112,None,92.3,NaN
228,147,FAPbI 3,None,Spiro-OMeTAD,NIP,test_2,ISOSLT,carbazole-triphenylamine and phenylammonium io...,None,None,1100,NaN,24.70,NaN,NaN,None,94.6,66.6
229,148,Cs0.05FA0.85MA0.10Pb(I0.97Br0.03)3,None,Spiro-OMeTAD,NIP,test_1_2,ISOSLT,phenylethylammonium iodide,50-70,None,500,NaN,NaN,NaN,NaN,None,84,70
230,148,Cs0.05FA0.85MA0.10Pb(I0.97Br0.03)3,None,Spiro-OMeTAD,NIP,test_1,None,4-tert-butyl-benzylammonium iodide,None,None,None,21.2,22.70,1.091,1.122,None,95,NaN


In [53]:
data.shape

(81, 18)

In [55]:
def fetch_smiles_from_name(molecule_name):
    try:
        # Search for the molecule in PubChem by name
        compounds = pcp.get_compounds(molecule_name, 'name')
        if compounds:
            return compounds[0].isomeric_smiles  # Return the first match's SMILES
        else:
            return np.nan
    except Exception as e:
        print(f"Error fetching SMILES for {molecule_name}: {e}")
        return None

In [57]:
import re

def fix_unmatched_brackets(s):
    """
    Fixes unmatched brackets in the given string by adding the correct brackets where necessary.

    :param s: Input string with potential unmatched brackets.
    :return: A corrected string with properly balanced brackets.
    """
    opening = "({["
    closing = ")}]"
    match = {')': '(', '}': '{', ']': '['}
    stack = []

    # Step 1: Identify missing closing brackets
    fixed_s = []
    for char in s:
        if char in opening:
            stack.append(char)
            fixed_s.append(char)
        elif char in closing:
            if stack and stack[-1] == match[char]:
                stack.pop()
                fixed_s.append(char)
            else:
                # Add missing opening bracket before unmatched closing
                fixed_s.insert(0, match[char])
                fixed_s.append(char)
        else:
            fixed_s.append(char)

    # Step 2: Add missing closing brackets at the end
    while stack:
        open_bracket = stack.pop()
        fixed_s.append(closing[opening.index(open_bracket)])

    return "".join(fixed_s)


def get_chemical_names(chemical_list):
    cleaned_list = []
    for name in chemical_list:
        # Remove text inside parentheses only if it's extra information (abbreviations)
        name = re.sub(r"\s*\([^)]*\)$", "", name).strip() 
        # Remove spaces after a closing bracket (ensure proper chemical formatting)
        name = re.sub(r"\] +", "]", name)

        cleaned_list.append(name)

    return cleaned_list

In [59]:
lst = data['passivating_molecule']
cleaned_list = lst.apply(lambda x: fix_unmatched_brackets(x))
cleaned_list = get_chemical_names(cleaned_list)
data['passivating_molecule_cleaned'] = cleaned_list
data['passivating_molecule_SMILES'] = data['passivating_molecule_cleaned'].apply(fetch_smiles_from_name)

/var/folders/_c/tmnx15c54v7cn_b4tx022flc0000gn/T/ipykernel_76121/4194259518.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['passivating_molecule_cleaned'] = cleaned_list
/var/folders/_c/tmnx15c54v7cn_b4tx022flc0000gn/T/ipykernel_76121/4194259518.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['passivating_molecule_SMILES'] = data['passivating_molecule_cleaned'].apply(fetch_smiles_from_name)


In [60]:
data['passivating_molecule_SMILES'].isna().sum()

47

In [79]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors

def compute_molecular_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            return [
                Descriptors.MolWt(mol),  # Molecular weight
                Descriptors.ExactMolWt(mol),  # Exact molecular weight (isotope-specific)
                Descriptors.MolLogP(mol),  # LogP (lipophilicity)
                Descriptors.TPSA(mol),  # Topological Polar Surface Area
                Descriptors.NumValenceElectrons(mol),  # Total valence electrons
                rdMolDescriptors.CalcNumRotatableBonds(mol),  # Rotatable bonds
                rdMolDescriptors.CalcNumHBA(mol),  # Hydrogen bond acceptors
                rdMolDescriptors.CalcNumHBD(mol),  # Hydrogen bond donors
                rdMolDescriptors.CalcFractionCSP3(mol),  # Fraction of sp3 carbons
                rdMolDescriptors.CalcNumAromaticRings(mol),  # Number of aromatic rings
                rdMolDescriptors.CalcNumSaturatedRings(mol),  # Number of saturated rings
                rdMolDescriptors.CalcNumHeteroatoms(mol),  # Number of heteroatoms
                rdMolDescriptors.CalcNumHeavyAtoms(mol),  # Number of heavy atoms
                rdMolDescriptors.CalcNumSpiroAtoms(mol),  # Number of spiro atoms
                rdMolDescriptors.CalcNumBridgeheadAtoms(mol),  # Number of bridgehead atoms
                Descriptors.FpDensityMorgan1(mol),  # Morgan fingerprint density (radius=1)
                Descriptors.FpDensityMorgan2(mol),  # Morgan fingerprint density (radius=2)
                Descriptors.FpDensityMorgan3(mol),  # Morgan fingerprint density (radius=3)
                Descriptors.qed(mol),  # Quantitative Estimate of Drug-likeness
                rdMolDescriptors.CalcNumLipinskiHBA(mol),  # Lipinski Hydrogen Bond Acceptors
                rdMolDescriptors.CalcNumLipinskiHBD(mol),  # Lipinski Hydrogen Bond Donors
                rdMolDescriptors.CalcNumRings(mol),  # Total number of rings
                rdMolDescriptors.CalcNumAmideBonds(mol),  # Number of amide bonds
                Descriptors.BalabanJ(mol),  # Balaban’s connectivity index
                Descriptors.BertzCT(mol),  # Bertz complexity
                Descriptors.Chi0(mol),  # Chi connectivity index (order 0)
                Descriptors.Chi1(mol),  # Chi connectivity index (order 1)
                Descriptors.Chi2n(mol),  # Chi connectivity index (order 2, non-H)
                Descriptors.Kappa1(mol),  # Kappa Shape Index (order 1)
                Descriptors.Kappa2(mol),  # Kappa Shape Index (order 2)
            ]
        else:
            return [np.nan] * 30  # Return NaN for missing values
    except:
        return [np.nan] * 30  # Return NaN for exceptions

# Apply function to dataset
mol_features = data['passivating_molecule_SMILES'].apply(compute_molecular_features)

# Convert list to DataFrame
mol_features_df = pd.DataFrame(mol_features.tolist(), 
                               columns=[
                                   'MolWt', 'ExactMolWt', 'LogP', 'TPSA', 'NumValenceElectrons',
                                   'NumRotBonds', 'NumHBA', 'NumHBD', 'FractionCSP3', 'AromaticRings',
                                   'SaturatedRings', 'Heteroatoms', 'HeavyAtoms', 'SpiroAtoms', 
                                   'BridgeheadAtoms', 'FpDensityMorgan1', 'FpDensityMorgan2', 
                                   'FpDensityMorgan3', 'QED', 'LipinskiHBA', 
                                   'LipinskiHBD', 'NumRings', 'NumAmideBonds', 'BalabanJ', 
                                   'BertzCT', 'Chi0', 'Chi1', 'Chi2n', 'Kappa1', 'Kappa2'
                               ],
                               index=data.index)

# Merge with original dataset
data = pd.concat([data, mol_features_df], axis=1)

In [71]:
data.to_csv('data_expanded_SMILES.csv', index=False)

In [87]:
data['passivating_molecule_SMILES'].isna().sum()

47